# LLM Saiga/Mistral 7B, Russian Mistral-based chatbot в DataSphereв DataSphere
Автор модели [IlyaGusev](https://huggingface.co/IlyaGusev).

На базе [Mistral OpenOrca](https://huggingface.co/Open-Orca/Mistral-7B-OpenOrca).


### Требования

1. 30 ГБ свободного места в хранилище проекта.
2. Минимальная конфигурация вычислительных ресурсов — g1.1 (1 карта GPU NVIDIA® Tesla® V100).
3. Токен доступа от huggingface.co, сохраненный в проекте в секрете `huggingface_key`.

### Устанавливаем библиотеки

После выполнения ячейки перезагрузите ядро (**Kernel -> Reset Kernel**).

In [ ]:
%pip install accelerate bitsandbytes peft transformers sentencepiece

### Задаем путь для сохранения кешированных данных модели

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/jupyter/datasphere/project/modelcache'

### Определяем токен для доступа к huggingface.co

In [ ]:
access_token = os.environ['huggingface_key']

### Определяем и скачиваем модель

In [ ]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<s>system\n"


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
config = PeftConfig.from_pretrained(MODEL_NAME, token=access_token)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, torch_dtype=torch.float32, device_map="auto", token=access_token)
model = PeftModel.from_pretrained(model, MODEL_NAME, torch_dtype=torch.float32, token=access_token)
model.eval()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False, token=access_token)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME, token=access_token)
#print(generation_config)

In [ ]:
from transformers import pipeline, Conversation

In [ ]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE,
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{"role": "system", "content": system_prompt}]

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

### Делаем запрос к модели

In [ ]:
inputs = ["Сочини рассказ, обязательно упоминая следующие объекты. Дано: дальняя дорога, поля, автомобиль и новые технологии", "Почему трава зеленая?"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(prompt)
    print(output)
    print()
    print("==============================")
    print()